# Lab. 2 Multi Layered Networks

### Ładowanie danych

PyTroch, a właściwie pakiet `torchvision` udostępnia parę przydatnych rzeczy, z których skorzystamy na dzisiejszych zajęciach.

Zacznijmy od ściąganie i ładowania danych, w [`torchvision.datasets`](https://pytorch.org/docs/stable/torchvision/datasets.html) znajdziemy popularne datasety, zajmiemy się dzisiaj MNISTem.

In [12]:
import torch
from torchvision.datasets import MNIST

train_data = MNIST(root='.', download=True, train=True)
test_data = MNIST(root='.', download=True, train=False)

train_data[0]

(<PIL.Image.Image image mode=L size=28x28 at 0x7F2AF2B06668>, 5)

Oprócz tego z samego `torcha` możemy skorzystać z [`DataLoadera`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader), który załatwia za nas sporo przydatnych rzeczy typu shufflowanie i batchowanie danych.

In [13]:
from torch.utils.data import DataLoader
from torchvision import transforms

datas = torch.div(train_data.data.type(torch.float), 255)
mi, st = datas.mean(), datas.std()

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[mi], std=[st], inplace=True),
    transforms.Lambda(lambda x: x.view(784))
])


train_data = [(img_transform(img[0]), img[1]) for img in train_data]
test_data = [(img_transform(img[0]), img[1]) for img in test_data]

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64)

for x, y in train_loader:
    print(x.shape)
    print(x.dtype)
    print(y)
    break

torch.Size([64, 784])
torch.float32
tensor([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9, 4, 0, 9, 1,
        1, 2, 4, 3, 2, 7, 3, 8, 6, 9, 0, 5, 6, 0, 7, 6, 1, 8, 7, 9, 3, 9, 8, 5,
        9, 3, 3, 0, 7, 4, 9, 8, 0, 9, 4, 1, 4, 4, 6, 0])


Wygląda na to, że aż tak bardzo za darmo wszystkiego nie dostaniemy, klasa `MNIST` zwraca nam dane w postaci obiektów [PILa](https://pillow.readthedocs.io/en/stable/). Musimy coś z tym zrobić.

## Zadanie 1.
1. Za pomocą [`transformerów`](https://pytorch.org/docs/stable/torchvision/transforms.html) przerobić powyższy kod tak aby zadziałał.  
**HINT**: sprawdzić jakie argumenty przyjmuje klasa `MNIST`.
2. Policzyć średnią i odchylenie standardowe wartości pojedynczego piksela dla całego zbioru trenującego i użyć ich do znormalizowania danych trenujących.  
**HINT**: Tutaj torchvision też powinien nam to ułatwić.
3. Zmienić "kształt" jednego przykładu z `28x28` na `784`.  
**HINT**: [`Lambda`](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.Lambda)

Uwaga: zwrócić uwagę co dokładnie robią używane _transformery_!

## Zadanie 2.

Ręcznie zaimplementować prostą sieć z jedną warstwą ukrtyą. Sieć ma mieć:
1. Jedną warstwę ukrytą rozmiaru 500 z wagami zainicjalizowanymi ze standardowego rozkładu normalnego.
2. Warstwa przy obu operacjach ma mieć uczone _biasy_ zainicjalizowane na 0.

**HINT**: Do rozkładu normalnego najlepiej użyć [`torch.randn`](https://pytorch.org/docs/stable/torch.html#torch.randn). Sprawdzić jakie ważne argumenty ta funkcja przyjmuje!

Należy oprócz tego zaimplementować pętlę uczenia z użyciem PyTorchowej funkcji kosztu _cross entropy_ i optymalizatora SGD.

In [0]:
from typing import List
import torch.nn as nn
import torch.nn.functional as F

class CustomNetwork(object):
    """
    Simple 1-hidden layer linear neural network
    """
    def __init__(self, batch_size, inp, hidd, out):
        """
        Initialize network's weights 
        """
        
        self.weight_1: torch.Tensor = torch.randn(inp, hidd, requires_grad=True)
        self.bias_1: torch.Tensor = torch.randn(batch_size, 1, requires_grad=True)
        self.bias_1.data.fill_(0)

        self.weight_2: torch.Tensor = torch.randn(hidd, out, requires_grad=True)
        self.bias_2: torch.Tensor = torch.randn(batch_size, 1, requires_grad=True)
        self.bias_2.data.fill_(0)
        
    def __call__(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass through the network
        """
        a: torch.Tensor = torch.mm(x, self.weight_1)
        a: torch.Tensor = a + self.bias_1.expand_as(a)
        a: torch.Tensor = torch.mm(a, self.weight_2)
        a: torch.Tensor = a + self.bias_2.expand_as(a)        
        return a
    
    def parameters(self) -> List[torch.Tensor]:
        """
        Returns all trainable parameters 
        """
        return [self.weight_1, self.bias_1, self.weight_2, self.bias_2]

In [16]:
from torch.optim import SGD
from torch.nn.functional import cross_entropy
from torch import optim

# some hyperparams
batch_size: int = 64
input_dim: int = 784
hidden_dim: int = 500
output_dim: int = 10
epoch: int = 3
lr: float = 0.01
momentum: float = 0.9

# prepare data loaders, base don the already loaded datasets
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

# initialize the model
model: CustomNetwork = CustomNetwork(batch_size, input_dim, hidden_dim, output_dim)

# initialize the optimizer
optimizer: torch.optim.Optimizer = optim.SGD(model.parameters(), lr, momentum)
  
# training loop
for e in range(epoch):
    for i, (x, y) in enumerate(train_loader):
        if x.shape[0] != batch_size: continue
        # reset the gradients from previouis iteration
        optimizer.zero_grad()
        # pass through the network
        output: torch.Tensor = model(x)
        # calculate loss
        loss: torch.Tensor = cross_entropy(output, y)
        # backward pass thorught the network
        loss.backward()
        # apply the gradients
        optimizer.step()
        
        # log the loss value
        if (i + 1) % 100 == 0:
            print(f"Epoch {e} iter {i+1}/{len(train_data) // batch_size} loss: {loss.item()}", end="\r")
            
    # at the end of an epoch run evaluation on the test set
    with torch.no_grad():
        # initialize the number of correct predictions
        correct: int = 0 
        for i, (x, y) in enumerate(test_loader):
            if x.shape[0] != batch_size: continue

            output: torch.Tensor = model(x)
            correct += (torch.max(output, 1)[1].view(batch_size) == y).sum().item()

        print(f"\nTest accuracy: {correct / len(test_data)}")

        
# this is your test
assert correct / len(test_data) > 0.8, "Subject to random seed you should be able to get >80% accuracy"


Test accuracy: 0.8276

Test accuracy: 0.8513

Test accuracy: 0.8434


## Zadanie 3.

1. Przepisać całą sieć do PyTorcha używając [`torch.nn.Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), [`torch.nn.Linear`](https://pytorch.org/docs/stable/nn.html#torch.nn.Linear).
2. Dodać [nieliniowe aktywacje](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity) i dodatkową warstwę, tak aby wyciągnąć przynajmniej 95% testowego accuracy w 3 epoki.

pu

In [0]:
class TorchNetwork(torch.nn.Module):
    """
    Simple 2-hidden layer non-linear neural network
    """
    def __init__(self, batch_size, input_dim, hidden_dim, hidden_dim2, output_dim):
        super(TorchNetwork, self).__init__()

        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.layer2 = nn.Linear(hidden_dim, hidden_dim2)
        self.layer3 = nn.Linear(hidden_dim2, output_dim)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [18]:
from torch.optim import SGD
from torch.nn.functional import cross_entropy

# some hyperparams
batch_size: int = 64
epoch: int = 3
lr: float = 0.01
momentum: float = 0.9
hidden_dim: int = 500
hidden_dim2: int = 200 
input_dim: int = 784
output_dim: int = 10

# prepare data loaders, base don the already loaded datasets
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

# initialize the model
model: TorchNetwork = TorchNetwork(batch_size, input_dim, hidden_dim, hidden_dim2, output_dim)
  
# initialize the optimizer
optimizer: torch.optim.Optimizer = optim.SGD(model.parameters(), lr, momentum)

# training loop
for e in range(epoch):
    tr_loss = 0
    te_loss = 0
    for i, (x, y) in enumerate(train_loader):
        model.zero_grad()
        output: torch.Tensor = model(x)
        loss: torch.Tensor = cross_entropy(output, y)
        loss.backward()
        optimizer.step()
        if (i + 1) % 100 == 0:
            print(f"Epoch {e} iter {i+1}/{len(train_data) // batch_size} loss: {loss.item()}", end='\r')
    # at the end of an epoch run evaluation on the test set
    with torch.no_grad():
        correct: int = 0
        for i, (x, y) in enumerate(test_loader):
            output: torch.Tensor = model(x)
            correct += (torch.max(output, 1)[1].view(y.shape[0]) == y).sum().item()       

        print(f"\nTest accuracy: {correct / len(test_data)}")
            
# this is your test
assert correct / len(test_data) > 0.95, "Subject to random seed you should be able to get >95% accuracy"


Test accuracy: 0.9559

Test accuracy: 0.9704

Test accuracy: 0.9731
